In [ ]:
import numpy as np
from numpy import genfromtxt

np.set_printoptions(suppress=True)
np.random.seed(0)

In [12]:
k = 2
alpha = 0.01
lamb = 0.1
iterations = 20

In [13]:
#import data
train_set = genfromtxt('Train.csv', delimiter=',')
train_set = train_set.astype(int)
train_set = np.delete(train_set, (0), axis=0)

In [14]:
#import data
users_table = genfromtxt('Users Demographics.csv', delimiter=',')
users_table = np.delete(users_table, (0), axis=0)
users = users_table[:,0]
movies_table = genfromtxt('Movies Metadata.csv', delimiter=',')
movies_table = np.delete(movies_table, (0), axis=0)
items = movies_table[:,0]

In [15]:
#initialize the biases
b_u = np.zeros(users.size) #user bias
b_i = np.zeros(items.size) #item bias
b = np.mean(train_set[:,2]) #global bias

In [16]:
def create_P_Q(k):
    #initialize user(P) and item(Q) feature matrix
    P = np.array(np.random.uniform(0, 1, users.shape[0] * k).reshape(users.shape[0], k))
    Q = np.array(np.random.uniform(0, 1, items.shape[0] * k).reshape(k, items.shape[0]))
    return P,Q

In [17]:
def train(train_set,P,Q,alpha,lamb,iterations):
    for i in range(iterations) :
        P,Q,mae = SGD(train_set, P, Q, alpha, lamb)
        print(f"iteration {i} : error is: {mae}")
        alpha = 0.9*alpha #update alpha
        
    return P,Q
    

In [18]:
def SGD(train_set, P, Q, alpha, lamb):
    #stochastic gradient descent
    total_error = 0
    for i,j,r in train_set:
        prediction = b + b_u[i-1] + b_i[j-1] + P[i-1,:].dot(Q[:,j-1])
        error = r - prediction
        
        #update biases
        b_u[i-1] = b_u[i-1] + alpha*(error-lamb*b_u[i-1])
        b_i[j-1] = b_i[j-1] + alpha*(error-lamb*b_i[j-1])
        
        #update matrrix
        P[i-1,:] = P[i-1,:] + alpha*(error*Q[:,j-1]-lamb*P[i-1,:])
        Q[:,j-1] = Q[:,j-1] + alpha*(error*P[i-1,:]-lamb*Q[:,j-1])
        
        total_error = total_error + abs(error)
        
    mae = total_error/train_set.shape[0] 
    return P,Q,mae


In [19]:
#def test(P,Q,validation_set):
#    יש לעבור שורה שורה בקובץ של הולידשיין ולבדוק מה הטעות בין הערך שנמצא שם לערך החזוי.
#    ולחשב את כל הטעויות האפשריות לפי מה שביקשו בתרגיל
    

In [20]:
#main
P,Q=create_P_Q(k)
train(train_set,P,Q,alpha,lamb,iterations)

iteration 0 : error is: 0.8678901215490492
iteration 1 : error is: 0.794195341316668
iteration 2 : error is: 0.770799801152177
iteration 3 : error is: 0.7585947445829343
iteration 4 : error is: 0.7510659074128101


(array([[ 0.28055961,  0.43381189],
        [ 0.43064592,  0.41820853],
        [ 0.43385675,  0.60070515],
        ...,
        [ 0.20412663,  0.11173261],
        [ 0.49390175, -0.10445662],
        [ 0.87417643,  0.85525087]]),
 array([[-0.00534643,  0.5271844 ,  0.28400577, ...,  0.64124793,
          0.37036376,  0.26510716],
        [-0.12190126,  0.3847558 ,  0.48642187, ...,  0.3868441 ,
          0.7029768 ,  0.31886146]]))

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
#rmse_val = rmse(np.array(d), np.array(p))

In [ ]:
def r_squre(predictions, targets):
    return 